In [1]:
import json

import sqlitedatastore as datastore
import solrindexer     as indexer
from annoutil import find_x_including_y


def load_sentence():
    data = []
    for doc_id in datastore.get_all_ids(limit=-1):
        row = datastore.get(doc_id, ['content', 'meta_info'])
        text = row['content']
        meta_info = json.loads(row['meta_info'])
        for i, sent in enumerate(datastore.get_annotation(doc_id, 'sentence')):
            # Solr へ登録するデータ構造へ変換
            data.append({
                'id':               '{0:d}.{1:s}.{2:d}'.format(doc_id, 'sentence', i),
                'doc_id_i':         doc_id,
                'anno_id_i':        i,
                'name_s':           'sentence',
                'sentence_txt_ja':  text[sent['begin']:sent['end']],
                'title_txt_ja':     meta_info['title'],
                'url_s':            meta_info['url'],
            })
    # Solr への登録を実行
    indexer.load('anno', data)


def load_affiliation():
    anno_name = 'affiliation'
    data = []
    for doc_id in datastore.get_all_ids(limit=-1):
        row = datastore.get(doc_id, ['content', 'meta_info'])
        text = row['content']
        meta_info = json.loads(row['meta_info'])
        sents = datastore.get_annotation(doc_id, 'sentence')
        for i, anno in enumerate(datastore.get_annotation(doc_id, anno_name)):
            # Solr へ登録するデータ構造へ変換
            sent = find_x_including_y(sents, anno)
            data.append({
                'id':                  '{0:d}.{1:s}.{2:d}'.format(doc_id, anno_name, i),
                'doc_id_i':            doc_id,
                'anno_id_i':           i,
                'name_s':              anno_name,
                'sentence_txt_ja':     text[sent['begin']:sent['end']],
                anno_name + '_txt_ja': text[anno['begin']:anno['end']],
                'title_txt_ja':        meta_info['title'],
                'url_s':               meta_info['url'],
            })
    # Solr への登録を実行
    indexer.load('anno', data)



# datastore.connect()
# load_sentence()
# load_affiliation()
# datastore.close()


{
  "responseHeader":{
    "status":0,
    "QTime":1633}}

{
  "responseHeader":{
    "status":0,
    "QTime":241}}

{
  "responseHeader":{
    "status":0,
    "QTime":22}}

{
  "responseHeader":{
    "status":0,
    "QTime":22}}



In [2]:
import json

import solrindexer as indexer


results = indexer.search(keywords=[['アメリカ'], ['大学']], rows=5)

print('responseHeader')
print(json.dumps(results['responseHeader'],
                 indent=4, ensure_ascii=False), '\n\n')

print('highlighting')
print(json.dumps(results['highlighting'],
                 indent=4, ensure_ascii=False), '\n\n')

print('response')
print(results['response']['numFound'])
for row in results['response']['docs']:
    for fl, value in row.items():
        if fl == 'content_txt_ja':
            value = value[:300].replace('\n', ' ')
        print('{0}\t{1}'.format(fl, value))
    print()


responseHeader
{
    "status": 0,
    "QTime": 655,
    "params": {
        "q": "(content_txt_ja:\"アメリカ\") AND (content_txt_ja:\"大学\")",
        "hl": "on",
        "hl.fl": "content_txt_ja",
        "rows": "5",
        "wt": "json"
    }
} 


highlighting
{
    "2": {
        "content_txt_ja": [
            "ブラジルは、南<em>アメリカ</em>に位置する連邦共和制国家。首都はブラジリア。\n南米大陸で最大の面積を占め、ウルグアイ、アルゼンチン、パラグアイ、ボリビア、ペルー、コロンビア、ベネズエラ、ガイアナ、スリナム、フランス領"
        ]
    },
    "12": {
        "content_txt_ja": [
            "に飛び地や諸権利を維持したのである。1492 - 93年の動きは、つまるところ<em>アメリカ</em>を開拓する準備であった。ネーデルラントからアキテーヌにかけて港が使えるようにしたフランスであったが、1494年"
        ]
    },
    "11": {
        "content_txt_ja": [
            "・リーグなどの<em>アメリカ</em>の<em>大学</em>側はインドに代表団を派遣して学生を集めるための事務所を構えたり、優秀なインド人学生をスカウトしたりするなどの活動もあり、<em>アメリカ</em>に留学するインド人学生は多く、移民税関捜査局"
        ]
    },
    "6": {
        "content_txt_ja": [
            "アメリカ合衆国<__EOS__>\nアメリカ合衆国（<em>アメリカ</em>がっしゅうこく、英: United States of America, USA）、通称<em>アメリカ</em>は、50の州および連邦区などからなる連邦共和"
     

In [9]:
import json

import solrindexer as indexer


results = indexer.search_annotation(
    fl_keyword_pairs=[
        ('affiliation_txt_ja', [['大学']])
    ],
    rows=5
)
print(json.dumps(results, indent=4, ensure_ascii=False))


{
    "responseHeader": {
        "status": 0,
        "QTime": 1,
        "params": {
            "q": "(affiliation_txt_ja:\"大学\")",
            "rows": "5",
            "wt": "json"
        }
    },
    "response": {
        "numFound": 27,
        "start": 0,
        "docs": [
            {
                "id": "1.affiliation.0",
                "doc_id_i": 1,
                "anno_id_i": 0,
                "name_s": "affiliation",
                "sentence_txt_ja": "フィジーにもキャンパスを設置する南太平洋大学もプロバイダを行っている",
                "affiliation_txt_ja": "南太平洋大学",
                "title_txt_ja": "フィジー",
                "url_s": "https://ja.wikipedia.org/wiki/%E3%83%95%E3%82%A3%E3%82%B8%E3%83%BC",
                "_version_": 1665406389353185280
            },
            {
                "id": "2.affiliation.3",
                "doc_id_i": 2,
                "anno_id_i": 3,
                "name_s": "affiliation",
                "sentence_txt_ja": "その後、独立してから1827年にサンパウロとオリンダ（のちにレシーフェに移転）に法科大学